<a href="https://colab.research.google.com/github/thuytran226/Prediction-of-UK-GDP-using-KNN/blob/main/Multivariate_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Import basic libraries 
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
## Import raw data
df_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/multivariate_data.csv') #insert path of data
df_raw.head()

# Data description
    # priv_con: private consumption
    # gov_ex : government expenditure
    # exp: exports
    # imp: imports
    # invest: investment rate
    # cpi: consumer price index
    # prod_i: production index
    # unemp: unemployment rate
    # rgdp: real gross domestic product 
    # gr_gdp: growth rate of gross domestic product

In [ ]:
## Convert the format of Time
df_raw['Time'] = pd.Series(df_raw['Time'], dtype='string')
df_raw['Time'] = df_raw['Time'].str.replace(' ','') 
df_raw.Year = pd.to_datetime(df_raw['Time'])
df_raw['Time'] = pd.PeriodIndex(df_raw.Year, freq='Q').to_timestamp()
df_raw['Time'] = df_raw['Time'] + pd.offsets.QuarterEnd(0)
# Set Time as index of data frame
df_raw= df_raw.set_index('Time')
df_raw.head()

In [ ]:
## Convert private consumption, government expenditure, exports, imports, and real GDP to log scale. 

df_raw['priv_con'] = np.log(df_raw['priv_con'])
df_raw['gov_ex'] = np.log(df_raw['gov_ex'])
df_raw['exp'] = np.log(df_raw['exp'])
df_raw['imp'] = np.log(df_raw['imp'])
df_raw['rgdp'] = np.log(df_raw['rgdp'])
df_raw

In [ ]:
## Take the first order of difference for the whole dataframe. 

df_trans = df_raw.diff(1).dropna()
df_trans = df_trans.rename(columns={'rgdp':'gr_gdp'}) # Note that except real gdp, the column names still remains unchanged to avoid the complexity. 
df_trans.head()

In [ ]:
## Test the stationary of time series

# Import the ADF function
from statsmodels.tsa.stattools import adfuller

# Perform ADF Test of each time series, in which the hypothesis H0: a unit root is present in time series; H1: no unit root exists
def adfuller_test(series, sig=0.05, name=''):
    res = adfuller(series, autolag='AIC')    
    p_value = round(res[1], 3) 

    if p_value <= sig:
        print(f' {name} : P-Value = {p_value} => Stationary. ')
    else:
        print(f'{name} : P-Value = {p_value} => Non-stationary.')

for name, column in df_trans.iteritems():
    adfuller_test(column, name=column.name)

In [ ]:
# Create a data frame which store all variables and their 4 lags
df1=df_trans.copy(deep=True)
for i in range(0, len(df1.columns)):
  for j in range(1, 5):
    df1[df1.columns[i] + '_lag_' + str(j)] =df1[[df1.columns[i]]].shift(j)

df1.head()

In [ ]:
## Arrange the data frame so it has the general form ( x(t-4)[i], x(t-3)[i], x(t-2)[i], x(t-1)[i], x[i], y(t-4), y(t-3), y(t-2), y(t-1), y) 
# with i is the independent variables. 

newnames=[]
for i in range(0, len(df_trans.columns)):
  for j in range(4,0,-1):
      newnames.append(df1.columns[len(df_trans.columns)-1+ i*4 + j])
  newnames.append(df_trans.columns[i])      
df2=df1[newnames]
df2.head()

In [ ]:
## Split the data with train/test ratio of 70/30:  70% for training and 30% for testing
# Hide this cell in the case of train/test 80/20

# df3:training data -> used for tuning hyperparameter and training the model
df3 = df2.iloc[0:round(0.7*len(df2['gr_gdp'])),:]

#df3: testing data -> used for perfoming out-of-sample forecasts
df4 = df2.iloc[round(0.7*len(df2['gr_gdp'])):len(df2['gr_gdp']) ,:]


In [ ]:
# ## Split the data with train/test ratio of 80/20:  80% for training and 20% for testing
# # Hide this cell in the case of train/test 70/30

# # df3:training data -> used for tuning hyperparameter and training the model
# df3 = df2.iloc[0:round(0.8*len(df2['gr_gdp'])),:]

# #df3: testing data -> used for perfoming out-of-sample forecasts
# df4 = df2.iloc[round(0.8*len(df2['gr_gdp'])):len(df2['gr_gdp']) ,:]


In [ ]:
## Create the feature subsets 

column_list = list(filter(lambda x: not x.endswith('gr_gdp'), df3.columns))
column_list
featuresubsets = []
featuresubsets.append(column_list)
for i in [4,3,2]:
  if i == 4:
    featuresubsets.append(list(filter(lambda x: not x.endswith(str(i)), column_list)))
  if i == 3: 
    featuresubsets.append(list(filter(lambda x: not x.endswith(str(i)) and not x.endswith(str(i+1)), column_list)))
  if i == 2:  
    featuresubsets.append(list(filter(lambda x: not x.endswith(str(i)) and not x.endswith(str(i+1)) and not x.endswith(str(i+2)), column_list)))
len(featuresubsets) 
featuresubsets

In [ ]:
## Perfom the Grid Search with 10-fold Cross-validation to tun the hyperparameters

# Import libraries and function for modelling

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from math import sqrt
import statistics

# Create a list to store the resulst of Gridsearch CV

FeatureNames = []
NumberOfFeatures = []
NumberOfNeighbors = []
WeightingScheme = []
DistanceMetric = []
ValidationAccuracy = []

# Separate the features and target values

X = df3.iloc[:,:-1]
Y = df3['gr_gdp']


# GRID SEARCH ALGORITHM

# loop throughs all feature subset
for num in range(0,len(featuresubsets)):
    # modify X by Xnew so that it includes only the selected feature subsets
    if type(featuresubsets[num]) == str:
        XNew = X[[featuresubsets[num]]].dropna()
    else:
        XNew = X[list(featuresubsets[num])].dropna()
        
    # For each feature subsets, create models with every values of K from 2 to 12. 
    for K in range(2,12):
        # For each combination of feature subsets and K, create models with different combinations of weighting schemes and distance metrics
        for weight in ['uniform', 'distance']:
            for distance in ['euclidean','manhattan']:
              
                model = KNeighborsRegressor(n_neighbors = K, weights = weight , metric = distance)
                
                # Creat a list to store the training/validation score of every iteration through Cross-validation process
                modeltrain = []
                modeltest = []
                
                # Perform the 10-folds cross-validation process, then store all training/validation scores of each iterations.
                cv = KFold(n_splits = 10,shuffle=True, random_state=123)
          
                for train,test in cv.split(XNew):
                    
                    xtrain,xtest,ytrain,ytest = XNew.iloc[train],XNew.iloc[test],Y.iloc[train],Y.iloc[test]
                    model.fit(xtrain,ytrain)
                    modeltest.append(model.score(xtest,ytest))
                    
                # Store the values of hyperparameters
                FeatureNames.append(featuresubsets[num])
                NumberOfFeatures.append(len(XNew.columns))
                NumberOfNeighbors.append(K)
                WeightingScheme.append(weight)
                DistanceMetric.append(distance)
                ValidationAccuracy.append(statistics.mean(modeltest)) # Take average validation score of all iterations of each model
              
# Create a summary report of Grid Search with 10-fold CV, then return the top 5 models with highest validation score. 
SummaryReport = pd.DataFrame()
SummaryReport['Feature Names'] = FeatureNames
SummaryReport['Number of Features'] = NumberOfFeatures
SummaryReport['Number of Neighbors K'] = NumberOfNeighbors
SummaryReport['Weighting Scheme'] = WeightingScheme
SummaryReport['Distance Metric'] = DistanceMetric
SummaryReport['Validation Accuracy'] = ValidationAccuracy
SummaryReport.sort_values('Validation Accuracy', ascending=False,inplace=True)
SummaryReport = SummaryReport.head(5)
SummaryReport.set_index('Feature Names',inplace = True)
SummaryReport


In [ ]:
## Create data for prediction based on the feature selection above

#DF for train
df_train = df3[['priv_con_lag_1','priv_con','gov_ex_lag_1','gov_ex','exp_lag_1','exp','imp_lag_1','imp','invest_lag_1','invest','prod_i_lag_1',
  'prod_i','cpi_lag_1','cpi','unemp_lag_1','unemp','gr_gdp_lag_1','gr_gdp']].dropna()
X_train = df_train.iloc[:,:-1]
Y_train = df_train['gr_gdp']

#DF for test

df_test = df_train = df4[['priv_con_lag_1','priv_con','gov_ex_lag_1','gov_ex','exp_lag_1','exp','imp_lag_1','imp','invest_lag_1','invest','prod_i_lag_1',
  'prod_i','cpi_lag_1','cpi','unemp_lag_1','unemp','gr_gdp_lag_1','gr_gdp']].dropna()
X_test = df_test.iloc[:,:-1]
Y_test = df_test['gr_gdp']



In [ ]:
## Perform out-of-sample forecast for one-step-ahead using KNN regression

# Import the accuracy metrics 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

# Create features and targets as input and output for training phase
X_train = df_train.iloc[:,:-1]
Y_train = df_train['gr_gdp']

# Create input for prediction, Y_test now is the observed values
X_test = df_test.iloc[:,:-1]
Y_test = df_test['gr_gdp']
    
# Perfom out-of-sample forecast

#Train/test ratio is 70/30 -> Hide this row if train/test ratio is 80/20
KNN = KNeighborsRegressor(n_neighbors = 4, weights = 'uniform', metric = 'manhattan')

#Train/test 80/20 -> Hide this row if train/test ratio is 70/30
# KNN = KNeighborsRegressor(n_neighbors = 8, weights = 'uniform', metric = 'manhattan')

knn = KNN.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)

# Print the forecast errors

print('MAE: ' + str(np.mean(np.abs(Y_test - Y_pred))))
print('RMSE: ' + str(np.sqrt(mse(Y_test, Y_pred))))

In [ ]:
## Create the data frame of results for visualizing a comparision of observed values and predicted values
df5=pd.DataFrame(Y_test)
df5['knn_pred'] = Y_pred.tolist()
df5=df5.sort_index()
df5

In [ ]:
## Plot the forecast comparision between observed values and predicted values in case of train/test 70/30
#Hide this cell in case of train/test 80/20

plt.figure(figsize=(9, 4))
plt.ylim(-0.04,0.04)
plt.plot(df5['gr_gdp'], 'b-', label = 'Actual')
plt.plot(df5['knn_pred'], 'r-', label = 'Predicted')
plt.xlabel('Year')
plt.ylabel('Log Scale')
plt.title('Multivariate KNN - Predicted vs. Actual Values (train/test = 70/30)')
plt.legend()
plt.show()

In [ ]:
# ## Plot the forecast comparision between observed values and predicted values in case of train/test 80/20
# #Hide this cell in case of train/test 70/30

# plt.figure(figsize=(9, 4))
# plt.ylim(-0.04,0.04)
# plt.plot(df5['gr_gdp'], 'b-', label = 'Actual')
# plt.plot(df5['knn_pred'], 'r-', label = 'Predicted')
# plt.xlabel('Year')
# plt.ylabel('Log Scale')
# plt.title('Multivariate KNN - Predicted vs. Actual Values (train/test = 80/20)')
# plt.legend()
# plt.show()

In [ ]:
## FORECASTING USING VAR MODEL

# Train/test ratio is 70/30 -> Hide the these two rows if  train/test ratio is 80/20
df_train = df_trans.iloc[0:round(0.7*len(df_trans)),:]
df_test = df_trans.iloc[round(0.7*len(df_trans)):len(df_trans),:]

# # # Train/test ratio is 80/20 -> Hide the these two rows if  train/test ratio is 70/30
# df_train = df_trans.iloc[0:round(0.8*len(df_trans)),:]
# df_test = df_trans.iloc[round(0.8*len(df_trans)):len(df_trans),:]

# Transform the data into stationary by taking the first order of difference of df_trans ( data frame of growth rate)
df_diff = df_train.diff(1).dropna()

# Perform ADF Test of each time series again, in which the hypothesis H0: a unit root is present in time series; H1: no unit root exists

for name, column in df_diff.iteritems():
    adfuller_test(column, name=column.name)

In [ ]:
#Import VAR model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(df_diff)
 
# Choose the order p of VAR model with maximum lag = 4 and criteria for model selection is AIC
for i in range(1,4):
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)

In [ ]:
# Another way of choosing the order p of VAR model with maximum lag = 4 and criteria for model selection is AIC
x = model.select_order(maxlags=4)
x.summary()

# p=4 is the optimal value

In [ ]:
# Fitting the model with order p=4
model_fit = model.fit(4)
# View the model summary
model_fit.summary()

In [ ]:
## Creat input data for forecasting
forecast_input = df_diff.values[-4:]
forecast_input

In [ ]:
# Make out-of-sample prediction

prediction = pd.DataFrame(model_fit.forecast(y= forecast_input, steps=len(df_test)),index = df_test.index, columns= df_train.columns+'1d')
prediction.head()

In [ ]:
## Since the predicted value now is a first-order difference, we need to invert the transfomation to the original scale. 
# De-difference of one order is taken by adding the original training data and a cumulative sum of predicted values

prediction['var_pred'] = df_train['gr_gdp'].iloc[-1] +  prediction['gr_gdp1d'].cumsum()
prediction['var_pred'] 



In [ ]:
# Add the predicted values from VAR to the data frame to compare with the actual values

df5['var_pred'] = prediction['var_pred'].tolist()
df5=df5.sort_index()
df5.head()

In [ ]:
# Print the forecast errors

print('MAE: ' + str(np.mean(np.abs(df5['gr_gdp'] - df5['var_pred']))))
print('RMSE: ' + str(np.sqrt(mse(df5['gr_gdp'], df5['var_pred']))))
